In [1]:
# https://dplyr.tidyverse.org/reference/context.html
%run nb_helpers.py
from datar.all import *

nb_header(cur_group_id, cur_group_rows, cur_data, cur_data_all, cur_column, book='context')

### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_group_id</div>

##### gives a unique numeric identifier for the current group.


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_group_rows</div>

##### Gives the row indices for the current group.

##### Args:
&emsp;&emsp;`_data`: The dataFrame.  

##### Returns:
&emsp;&emsp;The `_rows` from group data or row indexes  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_data</div>

##### gives the current data for the current group
(excluding grouping variables).  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_data_all</div>

##### gives the current data for the current group
(including grouping variables)  


### <div style="background-color: #EEE; padding: 5px 0 8px 0">★ cur_column</div>

##### Used in the functions of across. So we don't have to register it.


In [2]:
df = tibble(
  g=['a'] + ['b'] * 2 + ['c'] * 3,
  x=runif(6),
  y=runif(6)
)
gf = df >> group_by(f.g)

gf >> summarise(n = n())

g       n
  <object> <int64>
0        a       1
1        b       2
2        c       3

In [3]:
gf

g         x         y
  <object> <float64> <float64>
0        a  0.276842  0.639718
1        b  0.333175  0.149536
2        b  0.728975  0.400257
3        c  0.116262  0.087035
4        c  0.541848  0.312297
5        c  0.495865  0.114572
[TibbleGrouped: g (n=3)]

In [4]:
gf >> mutate(id=cur_group_id()) 

g         x         y      id
  <object> <float64> <float64> <int64>
0        a  0.276842  0.639718       0
1        b  0.333175  0.149536       1
2        b  0.728975  0.400257       1
3        c  0.116262  0.087035       2
4        c  0.541848  0.312297       2
5        c  0.495865  0.114572       2
[TibbleGrouped: g (n=3)]

In [5]:
gf >> summarise(row=cur_group_rows()) 

g        row
  <object>   <object>
0        a        [0]
1        b     [1, 2]
2        c  [3, 4, 5]

In [6]:
gf_group = gf >> summarise(data=cur_group())
gf_group 

g      data
  <object>  <object>
0        a  <DF 1x1>
1        b  <DF 1x1>
2        c  <DF 1x1>

In [7]:
gf_group >> pull(f.data)

0    <DF 1x1>
1    <DF 1x1>
2    <DF 1x1>
Name: data, dtype: object

In [8]:
gf_data = gf >> summarise(data=cur_data())
gf_data

g      data
  <object>  <object>
0        a  <DF 1x2>
1        b  <DF 2x2>
2        c  <DF 3x2>

In [9]:
gf_data >> pull(f.data, to='list')

[          x         y
   <float64> <float64>
 0  0.276842  0.639718,
           x         y
   <float64> <float64>
 1  0.333175  0.149536
 2  0.728975  0.400257,
           x         y
   <float64> <float64>
 3  0.116262  0.087035
 4  0.541848  0.312297
 5  0.495865  0.114572]

In [10]:
gf_data_all = gf >> summarise(data=cur_data_all())
gf_data_all

g      data
  <object>  <object>
0        a  <DF 1x3>
1        b  <DF 2x3>
2        c  <DF 3x3>

In [11]:
gf_data_all >> pull(f.data, to='list')

[         g         x         y
   <object> <float64> <float64>
 0        a  0.276842  0.639718,
          g         x         y
   <object> <float64> <float64>
 1        b  0.333175  0.149536
 2        b  0.728975  0.400257,
          g         x         y
   <object> <float64> <float64>
 3        c  0.116262  0.087035
 4        c  0.541848  0.312297
 5        c  0.495865  0.114572]

In [12]:
df >> select(f.x, f.y) >> mutate(
    across(
        everything(), 
        lambda x, cc: [cc + ' '] * x.shape[0] + (x**2).astype(str), cc=cur_column()
    )
)

x                       y
                 <object>                <object>
0   x 0.07664148433958724    y 0.4092387034410027
1   x 0.11100534084266422   y 0.02236109719928575
2    x 0.5314038334653144   y 0.16020604530995483
3  x 0.013516938415598222  y 0.007575071702234481
4    x 0.2935994793280256    y 0.0975293132771369
5   x 0.24588211859358594  y 0.013126717677172012

In [13]:
# or you can use x.name, since x is a Series
df >> mutate(across(
    [f.x, f.y], 
    lambda x: [x.name + ' '] * x.shape[0] + (x**2).astype(str)
))

g                       x                       y
  <object>                <object>                <object>
0        a   x 0.07664148433958724    y 0.4092387034410027
1        b   x 0.11100534084266422   y 0.02236109719928575
2        b    x 0.5314038334653144   y 0.16020604530995483
3        c  x 0.013516938415598222  y 0.007575071702234481
4        c    x 0.2935994793280256    y 0.0975293132771369
5        c   x 0.24588211859358594  y 0.013126717677172012